# Grover Adaptive Search and QUBOs

In this tutorial, we shall use a general version of [Grover's algorithm](./111_grover_en.ipynb) and [Quantum Amplitude Amplification](./110_amplitude_amplification_en.ipynb) to provide a quantum algorithm for a general QUBO problem. Note that this is a (complicated) alternative to the QAOA algorithms using gate-model quantum computing.

It is known that adiabatic quantum computing is equivalent to gate-model quantum computing. An important paper of Roland and Cerf [[1]] prove that the speedup of $O(\sqrt N)$ obtained via Grover search on an entire solution space of size $N$, is exactly the speedup of the QAOA algorithms.

In short, gate-based algorithms are theoretically equivalent to QAOA, but require coherent circuits with large number of qubits to set up and might not be practical to implement on a quantum computer in the near future.

[1]:https://arxiv.org/abs/quant-ph/0107015

In [1]:
# Install blueqat-sdk to get started.
# !pip install blueqat

## QUBO

A Quadratic Unconstrained Binary Optimization problem, in its most generality, asks to minimize 
$$X^TQX+RX,$$
where:
+ $Q$ is an $n\times n$ positive semidefinite matrix,
+ $R$ is a $1\times n$ row vector,
+ $X$ is an $n$-dimensional $\{0,1\}$-column vector.

_Note: A Constrained Quadratic Optimization Problem has additional constraints on the vectors $X$. It is well-known that (see [[2],[3]]) as long as these constraints are linear, such a problem can be reduced to a QUBO)._

Solving QUBO problems fast is a very important problem in modern computing. This is because several optimization problems such as Max-Cut, Graph Coloring, Vertex Cover etc. are QUBOs, as we've discussed in the earlier tutorials of this section.

[2]:https://arxiv.org/pdf/1912.04088.pdf
[3]:https://arxiv.org/pdf/1811.11538.pdf

## Grover's Algorithm vs. Grover Adaptive Search Algorithm (GAS)

In this tutorial, we shall be implementing [Gilliam-Woerner-Gonciulea's algorithm](https://arxiv.org/pdf/1912.04088.pdf) for $n=3$. But, before we discuss GAS, let us recall the basic idea behind Grover search.

### Grover's Algorithm
Grover search requires the following components:

+ An operator $A$ that maps $\lvert 0\rangle_n$ to the uniform superposition state $\lvert\varphi\rangle = \frac1{\sqrt{2^n}}\sum_{i}\lvert i\rangle_n$. In fact, $A=H^{\otimes n}$.
+ An oracle $O$ that _marks_ states $I\subseteq \{0,\ldots,2^n-1\}$, so that
$$O\lvert\varphi\rangle = \frac1{\sqrt{2^n}}\sum_{i\not\in I}\lvert i\rangle_n - \frac1{\sqrt{2^n}}\sum_{i\in I}\lvert i\rangle_n.$$
+ The diffusion operator $D$ that flips the sign of the state $\lvert 0\rangle_n$.

The Grover operator is $G=ADA^{\dagger}O$, which, when applied $\left\lfloor\frac{\pi}{4}\sqrt{\frac{2^n}{|I|}}\right\rfloor$ times on $A\lvert 0\rangle_n$, gives us a quantum state whose measurement is one of the entries of $I$ with high probability.

### Grover Adaptive Search Algorithm
The components required for implementing GAS are:

+ An operator $A_y$ such that
$$A_y\lvert 0\rangle_n\lvert 0\rangle_m = \frac 1{\sqrt{2^n}}\sum_{x} \lvert x\rangle_n\lvert f(x)-y\rangle_m.$$
+ An oracle $O_y$ such that
$$O_y\lvert x\rangle_n\lvert z\rangle_m = \text{sign}(z)\lvert x\rangle_n\lvert z\rangle_m.$$


**Input:** $f:X\to \mathbb R$, $\lambda >1$, num_iter (set to 3 by default).

**Algorithm:**
+ Uniformly sample $x_1\in X$ and set $y_1=f(x_1)$,
+ Set $k=1$ and $i=1$,
+ while(true):
    - Select $r_i\in\{0,\ldots,\lceil k-1\rceil\}$ uniformly at random
    - Apply Grover search with $r_i$ iterations, using oracles $A_{y_i}$ and $O$. Let the outputs be $x,y$.
        * If $y<y_i$, then set $x_{i+1}=x, y_{i+1}=y$, reset $k=1$.
        * Else set $x_{i+1}=x_i, y_{i+1}=y_i, k=\lambda k$.
    - i++
    - If the value of $x_i$ doesn't change for num_iter iterations, break.


## Implementation for a simple QUBO problem

Let us consider the following QUBO problem: $Q=\begin{pmatrix}1 & 0.5\\ 0.5&2\end{pmatrix}$, $R=\begin{pmatrix} 0 & -3\end{pmatrix}$. Then we are trying to minimize:
$$f(x_0,x_1) = x_0^2 + x_0x_1 + 2x_1^2 -3x_1 = x_0 +x_0x_1 -x_1.$$

The major difficulty in implementing our program is to construct a quantum circuit for the operators $A_y$ and $O$.

We shall use two registers:
+ A _key register_ with $n=2$ qubits corresponding to each variable in the problem, and 
+ A _value register_ with $m=2$ qubits to store the different values of $f(x_0, x_1)$, where the first qubit corresponds to the sign and the second qubit corresponds to the value.

For general QUBO problems, $m$ needs to be large enough so that $\max\limits_{x\in\{0,1\}^n} |f(x)| < 2^{m-1}$. In our case, $m=2$ will suffice!

### The operator $A_y$
This can be implemented using controlled phase operations.

+ We first create the uniform superposition state.
+ For a fixed $y$, $A_y$ needs to evaluate $f(x)-y$, which we will achieve by encoding $-y$ into the value register first: via phase gates with angles $\frac{-\pi y}{2}$ and $-\pi y$.
+ For the term $x_0$, we put controlled phase gates of angles $\frac{\pi}{2}$ and $\pi$ on the first and second qubits of the value register.
+ For the term $-x_1$, these angles will be $-\frac{\pi}{2}$ and $-\pi$.
+ For the term $x_0x_1$, we need two multi-controlled phase gates.
+ In general, for a term $ax_{i_1}x_{i_2}\cdots x_{i_k}$ in the polynomial, we put phase gates with controls on the $i_1,\ldots, i_k$'th qubits and angles $\frac{2\pi}{2^m}\cdot a\cdot 2^j$ on the $j$'th value register qubit.
+ Finally, after these controlled phase operations, we apply an inverse QFT to the value registers to compute $\lvert f(x)-y\rangle$ on the value register! Below is a circuit implementation for $A_y$:

![Circuit for A_1](./img/323_gas_ckt1.png)

We can reduce this circuit further to rewrite it in terms of 2-qubit gates. As 2-controlled phase operations can be decomposed into CPhase and CNOT gates, we obtain the following circuit:

![Circuit for A_1](./img/323_gas_ckt2.png)

We now implement this on blueqat. Recall the [QFT Circuit](./121_qft_en.ipynb) from our tutorials.

In [2]:
# Import Libraries
from blueqat import Circuit
import math

# Function to apply qft on a list of qubits of circuit
def apply_qft(circuit: Circuit(), qubits):
    num_qubits = len(qubits)
    for i in range(num_qubits):
        circuit.h[qubits[i]]
        for j in range(i+1, num_qubits):
            circuit.cphase(math.pi/(2 ** (j-i)))[qubits[j],qubits[i]] # Apply gate CR_{j-i}(qubit j, qubit i)
    # Reverse the order of qubits at the end
    for i in range(int(num_qubits/2)):
        circuit.swap(qubits[i],qubits[num_qubits-i-1])

# Function to apply iqft on a list of qubits of circuit
def apply_iqft(circuit: Circuit(), qubits):
    num_qubits = len(qubits)
    # Reverse the order of qubits
    for i in range(int(num_qubits/2)):
        circuit.swap(qubits[i],qubits[num_qubits-i-1])
    for i in range(num_qubits):
        for j in range(i+1, num_qubits):
            circuit.cphase(-math.pi/(2 ** (j-i)))[qubits[j],qubits[i]] # Apply gate CR_{j-i}(qubit j, qubit i)
        circuit.h[qubits[i]]

In [3]:
# Implementing the operator A_y for our function
def apply_Ay(circuit: Circuit(), yvalue: int):
    # Hadamard transform for Uniform superposition
    for i in range(4):
        circuit.h[i]
    # Encoding -y
    circuit.phase(-math.pi/2 * yvalue)[2]
    circuit.phase(-math.pi * yvalue)[3]
    # Encoding the polynomial
    # monomial: x0
    circuit.cphase(math.pi/2)[0,2]
    circuit.cphase(math.pi)[0,3]
    # monomial: -x1
    circuit.cphase(-math.pi/2)[1,2]
    circuit.cphase(-math.pi)[1,3]
    # monomial: x0*x1
    circuit.cphase(math.pi/4)[1,2]
    circuit.cx[1,0]
    circuit.cphase(-math.pi/4)[0,2]
    circuit.cx[1,0]
    circuit.cphase(math.pi/4)[0,2]
    circuit.cphase(math.pi/2)[1,3]
    circuit.cx[1,0]
    circuit.cphase(-math.pi/2)[0,3]
    circuit.cx[1,0]
    circuit.cphase(math.pi/2)[0,3]
    # IQFT
    apply_iqft(circuit, [2,3])

# A_y dagger:
def apply_Ay_dagger(circuit: Circuit(), yvalue: int):
    # QFT
    apply_qft(circuit, [2,3])
    # Inverting encoded polynomial
    # inverting monomial: x0
    circuit.cphase(-math.pi/2)[0,2]
    circuit.cphase(-math.pi)[0,3]
    # inverting monomial: -x1
    circuit.cphase(math.pi/2)[1,2]
    circuit.cphase(math.pi)[1,3]
    # inverting monomial: x0*x1
    circuit.cphase(-math.pi/4)[1,2]
    circuit.cx[1,0]
    circuit.cphase(math.pi/4)[0,2]
    circuit.cx[1,0]
    circuit.cphase(-math.pi/4)[0,2]
    circuit.cphase(-math.pi/2)[1,3]
    circuit.cx[1,0]
    circuit.cphase(math.pi/2)[0,3]
    circuit.cx[1,0]
    circuit.cphase(-math.pi/2)[0,3]
    # Inverting encoded -y
    circuit.phase(math.pi/2 * yvalue)[2]
    circuit.phase(math.pi * yvalue)[3]
    # Inverting Hadamard transform
    for i in range(4):
        circuit.h[i]

### The oracle O
As the oracle $O$ needs to flip the sign of the states which have $1$ in the first value register qubit signifying a negative sign, we only need to apply a $Z$ gate to implement this oracle.

In [4]:
def apply_oracle_Oy(circuit: Circuit()):
    circuit.z(2)

### The Grover Diffusion D
We need to implement the reflection around $\lvert 0000\rangle$ on blueqat-sdk. This can be achieved via the following circuit with an extra ancilla qubit.

![Diffusion-ancilla](./img/323_gas_ckt3.png)

In [5]:
def apply_diffusion(circuit: Circuit(), qubits, ancilla):
    if len(qubits) != 4:
        raise ValueError('Length of qubits must be 4')
    circuit.x(qubits)
    circuit.h[qubits[0]]
    circuit.ccx(qubits[2], qubits[3], ancilla)
    circuit.ccx(ancilla, qubits[1], qubits[0])
    circuit.ccx(qubits[2], qubits[3], ancilla)
    circuit.h[qubits[0]]
    circuit.x(qubits)

## Implementation of the main algorithm

Now we solve the optimization problem of minimizing $f(x_0,x_1) = x_0+x_0x_1-x_1$ on the blueqat simulator!

In [6]:
from random import randint
def evaluate_f(x0, x1):
    return x0 + x0*x1 -x1

# Initialize all variables
x0value = randint(0,1)
x1value = randint(0,1)
yvalue = evaluate_f(x0value, x1value)
print('Initial value: f({},{})={}'.format(x0value, x1value, yvalue))

k = 1
i = 1
lam = 8/7
t = 0

loops_without_improvement = 0

while True:
    rotation_count = randint(0,math.ceil(k-1))
    print('Creating Grover Circuit with {} rotations, k = {}, threshold = {}'.format(rotation_count, k, t))
    circuit = Circuit()
    apply_Ay(circuit, -t)
    for _ in range(rotation_count):
        apply_oracle_Oy(circuit)
        apply_Ay_dagger(circuit, -t)
        apply_diffusion(circuit, range(4), 4)
        apply_Ay(circuit, -t)
    circuit.m[:4]
    # print(circuit)
    result = circuit.run(shots=1000)
    # print(result)
    result_str = result.most_common(1)[0][0]
    # Get the most frequent output (x,y) from the circuit
    # print(result_str)
    x0_curr = int(result_str[0])
    x1_curr = int(result_str[1])
    y_curr = evaluate_f(x0_curr, x1_curr)
    print('Current value: f({},{})={}'.format(x0_curr, x1_curr, y_curr))
    if y_curr < yvalue:
        x0value = x0_curr
        x1value = x1_curr
        yvalue = y_curr
        t = y_curr
        k = 1.0
        circuit.reset()
        loops_without_improvement = 0
    else:
        k = math.ceil(lam * k)
        loops_without_improvement += 1

    print('Loops without improvement: {}'.format(loops_without_improvement))
    if loops_without_improvement == 5:
        break

print('Minimum Value is f({},{})={}'.format(x0value, x1value, yvalue))

Initial value: f(1,1)=1
Creating Grover Circuit with 0 rotations, k = 1, threshold = 0
Current value: f(1,0)=1
Loops without improvement: 1
Creating Grover Circuit with 0 rotations, k = 2, threshold = 0
Current value: f(1,0)=1
Loops without improvement: 2
Creating Grover Circuit with 1 rotations, k = 3, threshold = 0
Current value: f(1,0)=1
Loops without improvement: 3
Creating Grover Circuit with 3 rotations, k = 4, threshold = 0
Current value: f(0,1)=-1
Loops without improvement: 0
Creating Grover Circuit with 0 rotations, k = 1.0, threshold = -1
Current value: f(1,0)=1
Loops without improvement: 1
Creating Grover Circuit with 1 rotations, k = 2, threshold = -1
Current value: f(0,1)=-1
Loops without improvement: 2
Creating Grover Circuit with 2 rotations, k = 3, threshold = -1
Current value: f(1,0)=1
Loops without improvement: 3
Creating Grover Circuit with 0 rotations, k = 4, threshold = -1
Current value: f(1,1)=1
Loops without improvement: 4
Creating Grover Circuit with 3 rotations